In [1]:
!pip install numpy scikit-learn pandas kaggle

In [ ]:
import os
import json
import pandas as pd
from tqdm import tqdm
import numpy as np

In [2]:
df = pd.read_csv('train_ver2.csv')

/tmp/ipykernel_314619/1621394854.py:1: DtypeWarning: Columns (5,8,11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('train_ver2.csv')


In [3]:
# Loại bỏ những feature không cần thiết + lọc dữ liệu user của tháng 6/2015
def preprocess(df):
    df = df[df['fecha_dato'] == '2015-06-28']
    df = df.drop(columns=['fecha_dato', 'fecha_alta', 'ult_fec_cli_1t', 'tipodom', 'cod_prov'])
    
    # Convert the columns to numeric
    df[df.columns[-24:]] = df[df.columns[-24:]].apply(pd.to_numeric, errors='coerce')
    
    # Filter rows where the sum of the last 18 columns is greater than 0
    new_df = df.loc[df.iloc[:, -24:].sum(axis=1) > 0]
    
    return new_df

# Kết hợp dữ liệu user tháng 6/2015 và dữ liệu mua của tháng 5/2015
def lag_1_product(df):
    new_df = preprocess(df)
    user_ids = new_df['ncodpers'].unique()
    may_user_values = df[(df['ncodpers'].isin(user_ids)) & (df['fecha_dato'] == '2015-05-28')]

    shifted_1_value_product_per_user = []
    for user_id in tqdm(user_ids):
        user_values = may_user_values[may_user_values['ncodpers'] == user_id]
        last_24_values = user_values.iloc[:, -24:].values.flatten().tolist()
        shifted_1_value_product_per_user.append(last_24_values)
    new_df['shifted_1'] = shifted_1_value_product_per_user
    return new_df

train_df = lag_1_product(df)
train_df = train_df[train_df['shifted_1'].apply(len) == 24]

In [6]:
# Chuẩn hóa dữ liệu cho từng feature
def assert_dtype(df, dtype='test'):
    df.fillna(method='ffill', inplace=True)
    df = df[df['age']!=' NA']
    df['antiguedad'] = df['antiguedad'].apply(lambda x: pd.to_numeric(str(x).replace('[^\d.-]', ''), errors='coerce'))

    if dtype=='test':
        df = df.astype(dtype={'ind_actividad_cliente':"float", 'age': 'int',"indrel":"float", "ind_nuevo":"float" })
        return df
                                                                            
    df = df.astype(dtype={"ind_ahor_fin_ult1":"int8",'ind_aval_fin_ult1':"int8", 'ind_cco_fin_ult1':"int8", 
                                                                      'ind_cder_fin_ult1':"int8", 'ind_cno_fin_ult1':"int8", 'ind_ctju_fin_ult1':"int8", 'ind_ctma_fin_ult1':"int8", 
                                                                      'ind_ctop_fin_ult1':"int8", 'ind_ctpp_fin_ult1':"int8",
                                                                      'ind_nomina_ult1':"int8", 'ind_nom_pens_ult1':"int8",
                                                                      'ind_actividad_cliente':"float", 'age': 'int',
                                                                      "indrel":"float", "ind_nuevo":"float",
                                                                      'ind_deco_fin_ult1':"int8", 'ind_deme_fin_ult1':"int8", 'ind_dela_fin_ult1':"int8",
                                                                      'ind_ecue_fin_ult1':"int8", 'ind_fond_fin_ult1':"int8", 'ind_hip_fin_ult1':"int8",
                                                                      'ind_plan_fin_ult1':"int8", 'ind_pres_fin_ult1':"int8", 'ind_reca_fin_ult1':"int8", 'ind_tjcr_fin_ult1':"int8", 
                                                                      'ind_valo_fin_ult1':"int8", 'ind_viv_fin_ult1':"int8", 'ind_recibo_ult1':"int8"})
    return df

train_df = assert_dtype(train_df, dtype='train')

/tmp/ipykernel_314619/608064879.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)


In [8]:
categorical_col = ['ind_empleado', 'pais_residencia', 'sexo', 'ind_nuevo', 'tiprel_1mes', 'indresi', 'indext', 'conyuemp', 'canal_entrada', 'indfall', 'nomprov', 'ind_actividad_cliente', 'segmento']
interval_col = ['antiguedad', 'age', 'renta']
#Label col == các feature sẽ mua ở tháng 6
label_col = train_df.columns[-25:-1]

In [9]:
# Tạo các dict transform các feature
list_encoded = {}
for col in categorical_col:
    mapping_value = {}
    categorical = list(df[col].unique())
    for id, cate in enumerate(categorical):
        mapping_value[cate] = id
    list_encoded[col] = mapping_value

In [10]:
#Các function transform/encode các feature
def transform_age(age):
    if isinstance(age, str):
        return 0
    if age < 20:
        return 0
    elif age < 30:
        return 1
    elif age < 40:
        return 2
    elif age < 50:
        return 3
    elif age < 60:
        return 4
    else:
        return 5


def transform_categorical(dict_, x):
    if x not in dict_.keys():
        return -1
    return dict_[x]

def transform_membership(month):
    if isinstance(month, str):
        return 0
    if month < 24:
        return 0
    elif month < 60:
        return 1
    elif month < 120:
        return 2
    else: 
        return 3

def transform_income(income):
    if isinstance(income, str):
        return 0
    if income < 5e4:
        return 0
    elif income < 1e5:
        return 1
    elif income < 5e5:
        return 2
    else:
        return 3

def transform_value(df):
    for col in tqdm(categorical_col):
        df[col] = df[col].apply(lambda x: transform_categorical(list_encoded[col], x))
    df['age'] = df['age'].apply(lambda x: transform_age(x))
    df['antiguedad'] = df['antiguedad'].apply(lambda x: transform_membership(x))
    df['renta'] = df['renta'].apply(lambda x: transform_income(x)) 
    return df

train = transform_value(train_df)

100%|██████████| 13/13 [00:03<00:00,  3.95it/s]


In [11]:
# Phần xử lý dữ liệu tập test tương tự như tập Train
test = pd.read_csv('test_ver2.csv')

def preprocess(df):
    df = df.drop(columns=['fecha_dato', 'fecha_alta', 'ult_fec_cli_1t', 'tipodom', 'cod_prov'])
    return df

def lag_1_product(df, train_df):
    new_df = preprocess(df)
    user_ids = new_df['ncodpers'].unique()
    may_user_values = train_df[(train_df['ncodpers'].isin(user_ids)) & (train_df['fecha_dato'] == '2016-05-28')]

    shifted_1_value_product_per_user = []
    for user_id in tqdm(user_ids):
        user_values = may_user_values[may_user_values['ncodpers'] == user_id]
        last_24_values = user_values.iloc[:, -24:].values.flatten().tolist()
        shifted_1_value_product_per_user.append(last_24_values)
    new_df['shifted_1'] = shifted_1_value_product_per_user
    return new_df

test_lag_1 = lag_1_product(test, df)

/tmp/ipykernel_314619/2933419564.py:1: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  test = pd.read_csv('test_ver2.csv')
100%|██████████| 929615/929615 [11:56<00:00, 1296.84it/s]


In [12]:
# Concat các feature lại với nhau, mỗi sample training = 16 feature user information + 24 pass value purchase
def generate_dataset(df):
    X_1 = df[categorical_col].values
    X_2 = df[interval_col].values
    X_3 = np.array(df['shifted_1'].tolist())
    X = np.concatenate((X_1, X_2, X_3), axis=-1)
    if label_col[0] not in df.columns:
        return X
    y = df[label_col].values    
    assert X.shape[0] == y.shape[0]
    return X, y

Vì ta sẽ đưa dữ liệu vào một model dạng phân loại, mỗi feature training X sẽ dự đoán 1 sản phẩm tiếp theo, tuy nhiên theo dữ liệu ta có mỗi tháng 1 user có thể mua nhiều hơn 1 sản phẩm. Giả sử, 1 tháng A mua n sản phẩm => tách y thành n sample training, mỗi sample chỉ chứa 1 sản phẩm.

In [13]:
def remap_data(X, y):
    new_x, new_y = [], []
    x = X.tolist()
    for id, sample in tqdm(enumerate(y)):
        if sum(sample) > 0:
            indices = np.where(np.array(sample) == 1)[0]
            for idx in indices:
                new_x.append(X[id])
                new_y.append([int(idx)])
    assert len(new_x) == len(new_y)
    return np.array(new_x), np.array(new_y)

In [18]:
X, y = generate_dataset(train)
X, y = remap_data(X, y)
X_filled = np.nan_to_num(X, nan=-1) #Fill nan value
X_test = generate_dataset(transform_value(test_lag_1))
X_test[0]

610875it [00:09, 64053.49it/s]
100%|██████████| 13/13 [00:04<00:00,  2.67it/s]


array([ 4.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,  5.,  0., 18.,  0.,  3.,
        3.,  4.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  0.,  0.,  0.,
        0.])

In [28]:
from xgboost import XGBClassifier
bst = XGBClassifier(n_estimators=40, max_depth=50, learning_rate=1, objective='multi:softprob', device='cuda:1')
bst.fit(X=X_filled, y=y)
y_test = bst.predict(X_test)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device='cuda:1',
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=50, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=40, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [31]:
# Hàm xử lý dữ liệu inference sinh ra submit file
mapping_product = {}
for idx, product in enumerate(df.columns[-24:]):
    mapping_product[idx] = product
def get_submit_file(test_result):
    test_result = test_result.tolist()
    df = pd.DataFrame(zip(test['ncodpers'].tolist(), test_result), columns=['ncodpers', 'added_products'])
    df['added_products'] = df['added_products'].apply(lambda x: mapping_product[x])
    with open('submission.csv', 'w') as f:
        f.write('ncodpers,added_products')
        f.write('\n')
        for id in tqdm(range(len(df))):
            item = df.loc[id]
            f.write(f"""{item['ncodpers']},{item['added_products']}""")
            f.write('\n')
        f.close()

In [32]:
get_submit_file(y_test)

100%|██████████| 929615/929615 [00:36<00:00, 25183.27it/s]


In [57]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()
clf.fit(X_filled, y)
y_test = clf.predict(X_test)
get_submit_file(y_test)

/opt/conda/envs/bloom/lib/python3.9/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/envs/bloom/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
